In [ ]:
# AutoGluon Text 분석
# https://auto.gluon.ai/stable/tutorials/multimodal/text_prediction/beginner_text.html#sentiment-analysis-task

# Hyperparmeter 최적화
# https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html

In [ ]:
!pip3 install -U pip
!pip3 install -U setuptools wheel
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install autogluon
!pip install emoji
!pip install unidecode
!pip install Pillow==9.3.0
!pip install jedi==0.10
!pip install Pygments==2.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.
L

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.14.0
    Uninstalling Pygments-2.14.0:
      Successfully uninstalled Pygments-2.14.0


In [ ]:
#IMPORT
from google.colab import drive
import pandas as pd
import numpy as np

## 0. get files

In [ ]:
drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 전처리

### 1-1. remove null/nan

In [ ]:
# label info = 1의 수가 두배 이상 많음
print('1: ',len(train[train['AI']==1]))
print('0: ',len(train[train['AI']==0]))

1:  207
0:  539


In [ ]:
print(train.isna().sum())
print(test.isna().sum())

ID        1
prompt    0
answer    1
AI        2
dtype: int64
ID        0
prompt    0
answer    0
dtype: int64


In [ ]:
# col isna
for col in train.columns:
  # print(col,len(train[train[col].isna()]))
  display(train[train[col].isna()])

# null의 개수가 많지 않아, 삭제해도 될 것으로 판단
for col in train.columns:
  train = train[train[col].isna()==False]

,ID,prompt,answer,AI
610,NaN,0,NaN,NaN


,ID,prompt,answer,AI


,ID,prompt,answer,AI
610,NaN,0,NaN,NaN


,ID,prompt,answer,AI
609,785.0,define AI?,Artificial intelligence leverages computers an...,NaN
610,NaN,0,NaN,NaN


### 1-2. add wiki dataset

In [ ]:
import json
import os
curl = """
curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=Hello-SimpleAI%2FHC3&config=wiki_csai&split=train"
"""

In [ ]:
# run curl comand and load response --> json dict --> df
wiki_csai = json.loads(os.popen(curl).read())

In [ ]:


cols = [x['name'] for x in wiki_csai['features']]
rows = [x['row'].values() for x in wiki_csai['rows']]
wiki_data = pd.DataFrame(rows, columns=cols)

In [ ]:
wiki_df = pd.DataFrame([])
wiki_df['ID'] = [-1 for _ in range(len(wiki_data)*2)]
wiki_df['prompt'] = list(wiki_data['question'])*2
wiki_df['answer'] = [''.join(x) for x in wiki_data['human_answers']] + [''.join(x) for x in wiki_data['chatgpt_answers']]
wiki_df['AI'] = [0 for _ in range(len(wiki_data))] + [1 for _ in range(len(wiki_data))]
wiki_df

,ID,prompt,answer,AI
0,-1,"Please explain what is ""Animal cognition""",Animal cognition encompasses the mental capaci...,0
1,-1,"Please explain what is ""Human intelligence""",Human intelligence is the intellectual capabil...,0
2,-1,"Please explain what is ""Oxford English Diction...",The Oxford English Dictionary (OED) is the fir...,0
3,-1,"Please explain what is ""Oxford University Press""",Oxford University Press (OUP) is the universit...,0
4,-1,"Please explain what is ""AI applications""",Artificial intelligence (AI) has been used in ...,0
...,...,...,...,...
149,-1,"Please explain what is ""Optimization theory""",Optimization theory is a branch of mathematics...,1
150,-1,"Please explain what is ""Natural-language user ...",A natural-language user interface (NLUI) is a ...,1
151,-1,"Please explain what is ""Information retrieval""",Information retrieval (IR) is the process of s...,1
152,-1,"Please explain what is ""Question answering""",Question answering is a type of natural langua...,1


### 1-3. Text Cleaning: remove symbols

In [ ]:
import re
import emoji
from unidecode import unidecode

#remove html tags
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
#remove emoji and symbols
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("'re", ' are',x) # 're n't --> are not 
    x = re.sub("n't", ' not',x) # 're n't --> are not 
    x = re.sub(r'\([^)]*\)', '' ,x)
    x = re.sub("/n",' ',x) # index 3번 \r, \n 삭제
    x = re.sub("/r",' ',x) # index 3번 \r, \n 삭제
    x = re.sub(r'[^A-Za-z0-9!.?, ]', '', x) 
    # x = re.sub(r'[!.?]','/n',x) # removed to try sentence structure analysis
    x = re.sub(r'[,]',' ',x)
    # x = re.sub("@\w+", '',x) # removing mentions (@)
    # x = re.sub("'\d+", '',x)
    # x = re.sub("\d+", '',x)
    # x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    # x = re.sub("http\w+", '',x)
    # x = re.sub("\s[a-z]\s", '',x)
    x = re.sub("[ ]+",' ',x) # space*n --> space
    x = re.sub("_",'',x) # _people_ --> people
    x = x.strip()
    return unidecode(x) #emily bronté --> emily bronte, mi² --> mi2, πr --> pr

In [ ]:
train_text = train.copy()
train_text = pd.concat([train_text, wiki_df])
train_text["prompt"]=train["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train["answer"].apply(lambda x: dataPreprocessing(x))

In [ ]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [ ]:
print(len(train_text))
print(len(test_text))

900
249


In [ ]:
#check
for col in ['prompt', 'answer']:
  print([x for x in train_text[col] if len(list(filter(re.compile(r'[^A-Za-z0-9 ]').match,x)))>0])

['what is the future of ai?', 'what is your biggest challenge in your career?', 'what is the tallest mountain in the world?', 'what are the best 5 super cars?', 'what is deep learning?', 'what is the currency of the european union?', 'what is your opinion on the current state of renewable energy?', 'what is your favorite movie?', 'how do you succeed in digital marketing?', 'what is the most meaningful relationship in your life?', 'how do i become a millionaire?', 'what is the most meaningful thing in your life?', 'do you believe in ghosts?', 'how do i prevent scrum from turning great developers into average developers?', 'what is the best way to improve your selfesteem?', 'what is the best structure for a team ?', 'what are the most important skills for a technical career?', 'what is the importance of ai in healthcare?', 'is social media marketing more important than email marketing?', 'can you define friendly ai?', 'can you tell me the name of the tallest waterfall in the world?', 'wh

In [ ]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai?,the future of artificial intelligence is both ...,1.0
1,304.0,what is your biggest challenge in your career?,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world?,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best 5 super cars?,mclaren 720s.ferrari 296 gtb.audi r8.maserati ...,0.0
4,796.0,what is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
149,-1.0,what is the name of the largest rainforest in ...,the amazon rainforest,1.0
150,-1.0,what is the best way to handle change?,the best way to handle change is to stay posit...,1.0
151,-1.0,what is the history of ai?,the beginnings of modern ai can be traced to c...,1.0
152,-1.0,what is the difference between a static method...,a static method is a method that belongs to a ...,1.0


###1-4. sentence structure analysis
* AI 모델이 생성하는 문장은 사람이 생성하는 문장보다 훨씬 grammatically 정확할 것으로 생각된다.\
* PoS 분석을 통해 문장의 structure 를 분석, Noun - Verb - ... 이런식으로 구성된 변수를 생성해 활용. (AI의 문장이 훨씬 문법적으로 옳으며, 정형화 되어있을 것으로 예상)


In [ ]:
# 물음표가 있는 answer = human?
display(train_text[train_text['answer'].str.contains('\?')])
display(test_text[test_text['answer'].str.contains('\?')])

,ID,prompt,answer,AI
588,929.0,why is deep learning so easy?,if youre learning deep learning on your own th...,0.0


,ID,prompt,answer
13,988,how do i prevent scrum from turning great deve...,i think the problem in both your situation and...
74,959,are people who love fast cars against electric...,only paranoid retards are against things. are ...


In [ ]:
display(train_text[train_text['answer'].str.contains('\?')])
train_text[train_text['answer'].str.contains('\?')]['answer'].values

,ID,prompt,answer,AI
588,929.0,why is deep learning so easy?,if youre learning deep learning on your own th...,0.0


array(['if youre learning deep learning on your own then chances are youre following one of those online courses. ive been doing a few of those myself. heres why doing courses such as those might make deep learning seemingly easyfirstly a lot of these courses make use of frameworks like tensorflow or keras to implement neural networks and optimization algorithms like gradient descent. try implementing these with nothing but numpy or scipy. still find it easy? ill wait.a lot of the courses focus on an intuitive understanding of the mathematics rather than the actual hardcore stuff. i remember doing andrew ngs renowned ml course where in many cases the implication of several equations were shown as opposed to describing how they were derived. i found myself deriving some of them on my own just out of curiosity!'],
      dtype=object)

In [ ]:
# for p,a in zip(train_text['prompt'], train_text['answer']):
#   if 

### 1-5. train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

X = train_text.drop(['ID'], axis=1)
y = train_text['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pd.DataFrame(X_train, columns=['prompt', 'answer'])

,prompt,answer
10,how do i become a millionaire?,browse countless forums and youll find endless...
334,what is the most important lesson you have lea...,the most important lessons in life are subject...
244,are people who love fast cars against electric...,i know a lot of people who are seriously into ...
680,what are the most important skills for a digit...,the most important skills for a digital market...
306,what is the name of the deepest ocean trench i...,the deepest trench in the ocean is the mariana...
...,...,...
106,what is the currency of australia?,australian dollar.
270,what is the best way to stay positive?,the best way to stay positive in my opinion is...
114,what is your biggest weakness?,my biggest weakness is my tendency to be too h...
435,what is the name of the largest underwater cav...,sac actun cave system


In [ ]:
# pd.m(y_train, columns=['AI'])

In [ ]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,prompt,answer,AI
10,how do i become a millionaire?,browse countless forums and youll find endless...,0.0
334,what is the most important lesson you have lea...,the most important lessons in life are subject...,1.0
244,are people who love fast cars against electric...,i know a lot of people who are seriously into ...,0.0
680,what are the most important skills for a digit...,the most important skills for a digital market...,0.0
306,what is the name of the deepest ocean trench i...,the deepest trench in the ocean is the mariana...,0.0
...,...,...,...
106,what is the currency of australia?,australian dollar.,0.0
270,what is the best way to stay positive?,the best way to stay positive in my opinion is...,0.0
114,what is your biggest weakness?,my biggest weakness is my tendency to be too h...,1.0
435,what is the name of the largest underwater cav...,sac actun cave system,0.0


In [ ]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['prompt', 'answer']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,prompt,answer,AI
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt.,1.0
81,what is the most important factor in making a ...,the most important factor in making a decision...,1.0
231,how do i concatenate strings in python?,you can concatenate strings in python by using...,0.0
588,why is deep learning so easy?,if youre learning deep learning on your own th...,0.0
39,what are the most important life skills to learn?,the most important life skills to learn includ...,0.0
...,...,...,...
151,what is the history of ai?,the beginnings of modern ai can be traced to c...,1.0
578,what is the history of ai?,the history of ai can be traced back to ancien...,1.0
33,can you give me the name of the largest saltwa...,the largest saltwater lake in the world is the...,0.0
25,what is the best way to stay positive?,the best way to stay positive in my opinion is...,0.0


## 2. 모델링

In [ ]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 7

In [ ]:
from ray import tune

In [ ]:
hyperparameters = {
        # "optimization.learning_rate": tune.uniform(0.00005, 0.005),
        # "optimization.optim_type": tune.choice(["adamw", "sgd"]),
        # "optimization.max_epochs": tune.choice(["40", "50"]),
        # "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"])
}
hyperparameter_tune_kwargs = {
    # "searcher": "bayes", # random
    # "scheduler": "ASHA",
    # "num_trials": 2,
}

In [ ]:
# 메모리 이슈 해결
import os
import ray

# Set the memory usage threshold to adjust the kill threshold
# os.environ['RAY_memory_usage_threshold'] = str(1 * 1024 * 1024 * 1024)  # 1GB

# Disable worker killing to prevent OOM errors
# os.environ['RAY_memory_monitor_refresh_ms'] = str(0)  # 0ms

# Initialize Ray with additional resources
# ray.init(memory=10 * 1024 * 1024 * 1024, object_store_memory=10 * 1024 * 1024 * 1024, num_cpus=8)

In [ ]:
from autogluon.multimodal import MultiModalPredictor

# 모듈 못찾는 Attribute Error 발생시 런타임 초기화 후 코드 처음부터 재실행
# EX. AttributeError: module 'PIL.Image' has no attribute 'Resampling'
import uuid

# model_path = PATH+f"model/{uuid.uuid4().hex}-automm_sst_QA_tuning_6hours"
# predictor = MultiModalPredictor(label=label, eval_metric=eval_metric, path=model_path)
predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)

predictor.fit(train_data, presets='best_quality', 
              hyperparameters={"optimization.max_epochs": 20, "optimization.patience": 30},
              time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20230307_085034/"
AutoMM starts to create your model. ✨

- Model will be saved to "/content/AutogluonModels/ag-20230307_085034".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20230307_085034
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 100: 'val_acc' was not in top 3
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20230307_085034")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20230307_085034
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




## 3. 성능평가

In [ ]:
# 모델 성능평가
# predictor.leaderboard(silent = True)

In [ ]:
test_score = predictor.evaluate(test_data)
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.75}


In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_text)

Predicting: 0it [00:00, ?it/s]

In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,0
...,...,...
244,702,0
245,500,0
246,818,1
247,584,0


In [ ]:
submission.to_csv(PATH+'submit/autoGluon_QA_epoch20_cha_230307.csv', index=False)

In [ ]:
MODELNAME = "autoGluon_QA_epoch20_cha_230307"
model = predictor.dump_model()

import pickle

# 모델 객체 저장
with open(PATH+'model/'+MODELNAME+'.pkl', 'wb') as f:
    pickle.dump(model, f)

from joblib import dump, load

# 모델 객체 저장
dump(model, PATH+'model/'+MODELNAME+'.joblib')

# 저장된 모델 객체 로드
# model = load('model.joblib')

Model weights for hf_text are saved to /content/AutogluonModels/ag-20230307_085034/hf_text.
Tokenizer hf_text saved to /content/AutogluonModels/ag-20230307_085034/hf_text.


['/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/autoGluon_QA_epoch20_cha_230307.joblib']